In [8]:
from utils import start_tensorboard

start_tensorboard()

📊 TensorBoard Setup Instructions

1. Open a terminal in Azure ML Studio

2. Run this command:

   tensorboard --logdir /mnt/batch/tasks/shared/LS_root/mounts/clusters/ml-ais-p29636/code/Users/p29636/LectureNotes/exercise_8/logs --host 0.0.0.0 --port 6006

3. Open this URL in your browser:

   https://ml-ais-p29636-6006.swedencentral.instances.azureml.ms



'https://ml-ais-p29636-6006.swedencentral.instances.azureml.ms'

In [9]:
import shutil
import os

if os.path.exists("logs"):
    shutil.rmtree("logs")
    print("Logs directory cleared")
else:
    print("No logs directory found")

Logs directory cleared


# Introduction to Keras and layer types
Previously you built a simple artificial neural net (ANN) with one hidden layer and sigmoid activation function. Today we will take a look at a few adaptations of such a basic net both in terms of NN architecture as well as layer types. We will still stay with Fully Connected Neural Networks for this first exercise!

things to go through:
- activation functions (issue with sigmoid -> ReLU, Leaky ReLU)
- drop out
- skip connections
- deeper neural nets
- potential issues of Fully Connected Neural Networks (Scaling?)
- Keras things like functional API

## Setup things
Make sure your environment has all the required packages available. Take care to have
- keras (Read [This short guide](https://keras.io/getting_started/))
- a backend of your choosing for Keras (I dont care which one you use but we will stay with Tensorflow for now)
- scikit-learn
- pandas
- tensorboard (optional)

Feel free to use a package manager of your choice (avoid conda) or a premade environment in Azure.
Also make sure you have the data files you need ready, I recommend putting them into a ./data/ subdir

## Loading and preparing the dataset


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("data/energydata_complete.csv")

# For timeseries such as this, are rows really independent?
# Is there a separate approach to this problem? What patterns do we lose by treating the dataset like this?
df['date_time'] = pd.to_datetime(df['date'])

df['dayinweek'] = df['date_time'].dt.day_of_week
df['month'] = df['date_time'].dt.month
df['hour'] = df["date_time"].dt.hour
df['minutes'] = df['date_time'].dt.minute
df.drop(columns=['date_time'], inplace=True)

# how do we keep it random but reproducible?

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["Appliances","date"]), df['Appliances'],random_state=42)
print(X_train.shape, X_test.shape)

sd = StandardScaler()
X_train_scaled = sd.fit_transform(X_train)
X_test_scaled = sd.transform(X_test)
X_train.head(3)

## Vanishing Gradient Problem with Sigmoid

Sigmoid's derivative ranges from 0 to 0.25 (max at the middle).

During backpropagation, gradients multiply layer-by-layer: 0.25 × 0.25 × 0.25...

**Result:** Gradients shrink exponentially in deeper networks → early layers barely learn. **Deeper Networks fail with sigmoid**

**Solution:** Use e.g. ReLU (derivative = 1 for positive inputs).

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
print(f"Using Keras {keras.__version__} with backend: {keras.backend.backend()}")

Using Keras 3.12.0 with backend: tensorflow


In [12]:
import keras
from utils import train_model,eval_model

model_sigmoid = keras.models.Sequential([
    keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(512, activation="sigmoid"),
    keras.layers.Dense(256, activation="sigmoid"),
    keras.layers.Dense(128, activation="sigmoid"),
    keras.layers.Dense(64, activation="sigmoid"),
    keras.layers.Dense(32, activation="sigmoid"),
    keras.layers.Dense(1, activation=None)
])

model_sigmoid.compile(optimizer="adam", loss="mse", metrics=["mae"])
model_sigmoid.summary()

I0000 00:00:1764280427.872916   63849 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14793 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0001:00:00.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 190,977 (746.00 KB)

 Trainable params: 190,977 (746.00 KB)

...

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 190,977 (746.00 KB)

 Trainable params: 190,977 (746.00 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model_relu = keras.models.Sequential([
    keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(1, activation=None)
])

model_relu.compile(optimizer="adam", loss="mse", metrics=["mae"])
model_relu.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 512)                 │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 190,977 (746.00 KB)

 Trainable params: 190,977 (746.00 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model_leaky_relu = keras.models.Sequential([
    keras.layers.Input(shape=(X_train_scaled.shape[1],)), #
    keras.layers.Dense(512),
    keras.layers.LeakyReLU(negative_slope=0.01),
    keras.layers.Dense(256),
    keras.layers.LeakyReLU(negative_slope=0.01),
    keras.layers.Dense(128),
    keras.layers.LeakyReLU(negative_slope=0.01),
    keras.layers.Dense(64),
    keras.layers.LeakyReLU(negative_slope=0.01),
    keras.layers.Dense(32),
    keras.layers.LeakyReLU(negative_slope=0.01),
    keras.layers.Dense(1, activation=None)
])

model_leaky_relu.compile(optimizer="adam", loss="mse", metrics=["mae"])
model_leaky_relu.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 512)                 │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 190,977 (746.00 KB)

 Trainable params: 190,977 (746.00 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
history_sigmoid = train_model(model_sigmoid, X_train_scaled, y_train, "sigmoid_deep")
history_relu = train_model(model_relu, X_train_scaled, y_train, "relu_deep")
history_leaky_relu = train_model(model_leaky_relu, X_train_scaled, y_train, "leaky_relu_deep")

Epoch 1/50


2025-11-27 22:09:45.406779: I external/local_xla/xla/service/service.cc:163] XLA service 0x7c68980021d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-27 22:09:45.406800: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-11-27 22:09:45.440129: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-27 22:09:45.643299: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002
I0000 00:00:1764281386.705564   71537 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


370/370 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 18905.9238 - mae: 90.1935 - val_loss: 17792.7168 - val_mae: 85.5756
Epoch 2/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 17521.7070 - mae: 82.1685 - val_loss: 16584.0410 - val_mae: 78.1953
Epoch 3/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 16329.5166 - mae: 74.7332 - val_loss: 15428.9629 - val_mae: 70.6504
Epoch 4/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 15280.3652 - mae: 67.6737 - val_loss: 14482.7031 - val_mae: 64.0867
Epoch 5/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 14402.9268 - mae: 61.5466 - val_loss: 13678.1768 - val_mae: 58.2614
Epoch 6/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 13658.9219 - mae: 56.5608 - val_loss: 12999.1885 - val_mae: 53.8549
Epoch 7/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 13029.8369 - mae: 52.7962 - val_loss: 12426.6650 - val_mae: 51.1931
Epoch 8/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 12503.7959 - mae: 51.0921 - val_loss: 11954.1826 - val_

In [16]:
# eval on test set
from utils import eval_model

eval_model(model_sigmoid, X_test_scaled, y_test, "Sigmoid Deep Network")
eval_model(model_relu, X_test_scaled, y_test, "ReLU Deep Network")
eval_model(model_leaky_relu, X_test_scaled, y_test, "Leaky ReLU Deep Network")


  Sigmoid Deep Network - Test Results
  Test Loss (MSE): 9,096.35
  Test MAE:        51.21
  R² Score:        0.0843 (8.43% variance explained)


  ReLU Deep Network - Test Results
  Test Loss (MSE): 6,663.43
  Test MAE:        41.37
  R² Score:        0.3292 (32.92% variance explained)


  Leaky ReLU Deep Network - Test Results
  Test Loss (MSE): 6,699.45
  Test MAE:        40.24
  R² Score:        0.3256 (32.56% variance explained)



(6699.4541015625, 40.236915588378906, 0.32559388875961304)

## Dropout - Preventing Overfitting

Deep networks can **memorize trained on data** instead of learning generalizations.

**Dropout** randomly sets a percentage of neurons to 0 during each training step (e.g., 30% dropout → 30% of neurons turned off).

This forces the network to learn **robust features** that don't rely on a specific small subset of neurons. We will see on the tensorboard curves, that the model is learning slower but also less likely to overfit.


In [17]:
## Dropout 
model_with_dropout = keras.models.Sequential([
    keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation=None)
])

model_with_dropout.compile(optimizer="adam", loss="mse", metrics=["mae"])
model_with_dropout.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                     │ (None, 512)                 │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 190,977 (746.00 KB)

 Trainable params: 190,977 (746.00 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
history_with_dropout = train_model(model_with_dropout, X_train_scaled, y_train, "with_dropout", epochs=50)

Epoch 1/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 10667.4883 - mae: 57.7650 - val_loss: 8744.7461 - val_mae: 52.0763
Epoch 2/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9528.4395 - mae: 53.6257 - val_loss: 8216.5400 - val_mae: 50.7269
Epoch 3/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9182.0088 - mae: 52.7087 - val_loss: 8158.2769 - val_mae: 47.2453
Epoch 4/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8967.9609 - mae: 51.8663 - val_loss: 7983.6899 - val_mae: 46.3059
Epoch 5/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8689.7510 - mae: 50.9664 - val_loss: 8177.1211 - val_mae: 44.4632
Epoch 6/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8571.1123 - mae: 49.9821 - val_loss: 7726.0967 - val_mae: 43.7747
Epoch 7/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8598.4355 - mae: 49.7091 - val_loss: 7930.4092 - val_mae: 42.2491
Epoch 8/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8302.1387 - mae: 48.7183 - val_loss: 7535.9712 - val_mae:

In [19]:
eval_model(model_with_dropout, X_test_scaled, y_test, "ReLU with Dropout")


  ReLU with Dropout - Test Results
  Test Loss (MSE): 6,357.82
  Test MAE:        38.57
  R² Score:        0.3600 (36.00% variance explained)



(6357.82177734375, 38.56818771362305, 0.35998475551605225)

# Batchnorm
**Problem:** During training, the distribution of inputs to each layer changes as previous layers update (internal covariate shift), slowing down training.

**Batch Normalization** normalizes layer inputs by computing mean and standard deviation across the batch, then scaling and shifting with learned parameters.

Batchnorm is quite optional, at the end the effect on performance should be tested empirically. 


In [20]:
model_with_batchnorm = keras.models.Sequential([
    keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(512),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(256),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation=None)
])

model_with_batchnorm.compile(optimizer="adam", loss="mse", metrics=["mae"])
model_with_batchnorm.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                     │ (None, 512)                 │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 32)                  │              

 Total params: 194,945 (761.50 KB)

 Trainable params: 192,961 (753.75 KB)

 Non-trainable params: 1,984 (7.75 KB)

In [21]:
history_with_batchnorm = train_model(model_with_batchnorm, X_train_scaled, y_train, "with_dropout_and_batchnorm", epochs=50)

Epoch 1/50


370/370 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 19403.0508 - mae: 94.5128 - val_loss: 17446.1230 - val_mae: 88.7322
Epoch 2/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 15753.7383 - mae: 79.6588 - val_loss: 12857.8252 - val_mae: 68.9786
Epoch 3/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 11730.2012 - mae: 61.7536 - val_loss: 9823.0029 - val_mae: 54.6788
Epoch 4/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9404.8359 - mae: 53.2744 - val_loss: 8106.6465 - val_mae: 49.3284
Epoch 5/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8613.3086 - mae: 50.4975 - val_loss: 7651.6528 - val_mae: 48.0364
Epoch 6/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8275.4756 - mae: 49.1456 - val_loss: 7563.3867 - val_mae: 45.8934
Epoch 7/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 8229.8955 - mae: 49.3542 - val_loss: 7457.6064 - val_mae: 43.2028
Epoch 8/50
370/370 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8097.2744 - mae: 48.7700 - val_loss: 7416.2681 - val_mae: 43.229

In [22]:
eval_model(model_with_batchnorm, X_test_scaled, y_test, "ReLU with Dropout and Batchnorm")


  ReLU with Dropout and Batchnorm - Test Results
  Test Loss (MSE): 6,297.74
  Test MAE:        39.28
  R² Score:        0.3660 (36.60% variance explained)



(6297.74365234375, 39.2813720703125, 0.3660328984260559)


## Keras Functional API

The **Sequential API** is simple but limited - layers stack linearly, no branching or multiple inputs/outputs. It is quite rigid in how we can define and interact with the architecture.

The **Functional API** is more flexible (and follows typical functional programming styles - think lambda):
- Multiple inputs/outputs
- Layer sharing
- Branching and merging
- Skip connections (residual networks)
- also it allows us to dynamically construct different architectures e.g. for hyperparam search.


In [24]:
# Functional API version of dropout model
inputs = keras.layers.Input(shape=(X_train_scaled.shape[1],))

x = keras.layers.Dense(512, activation="relu")(inputs)
x = keras.layers.Dropout(0.3)(x)

x = keras.layers.Dense(256, activation="relu")(x)
x = keras.layers.Dropout(0.3)(x)

x = keras.layers.Dense(128, activation="relu")(x)
x = keras.layers.Dropout(0.3)(x)

x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.Dropout(0.2)(x)

x = keras.layers.Dense(32, activation="relu")(x)
x = keras.layers.Dropout(0.2)(x)

outputs = keras.layers.Dense(1, activation=None)(x)

model_functional = keras.Model(inputs=inputs, outputs=outputs, name="functional_dropout")
model_functional.compile(optimizer="adam", loss="mse", metrics=["mae"])
model_functional.summary()

Model: "functional_dropout"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 31)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 512)                 │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_32 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 190,977 (746.00 KB)

 Trainable params: 190,977 (746.00 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Your turn, implement a skip connection from the first dense+dropout layer block to the last (excluding output layer)